# Lab 2: Inference in Graphical Models

### Machine Learning 2 (2017/2018)

* The lab exercises should be made in groups of two people.
* The deadline is Thursday, 29.04, 23:59.
* Assignment should be submitted through BlackBoard! Make sure to include your and your teammates' names with the submission.
* Attach the .IPYNB (IPython Notebook) file containing your code and answers. Naming of the file should be "studentid1\_studentid2\_lab#", for example, the attached file should be "12345\_12346\_lab1.ipynb". Only use underscores ("\_") to connect ids, otherwise the files cannot be parsed.

Notes on implementation:

* You should write your code and answers in an IPython Notebook: http://ipython.org/notebook.html. If you have problems, please ask.
* Use __one cell__ for code and markdown answers only!
    * Put all code in the cell with the ```# YOUR CODE HERE``` comment and overwrite the ```raise NotImplementedError()``` line.
    * For theoretical questions, put your solution using LaTeX style formatting in the YOUR ANSWER HERE cell.
* Among the first lines of your notebook should be "%pylab inline". This imports all required modules, and your plots will appear inline.
* Large parts of you notebook will be graded automatically. Therefore it is important that your notebook can be run completely without errors and within a reasonable time limit. To test your notebook before submission, select Kernel -> Restart \& Run All.

### Introduction
In this assignment, we will implement the sum-product and max-sum algorithms for factor graphs over discrete variables. The relevant theory is covered in chapter 8 of Bishop's PRML book, in particular section 8.4. Read this chapter carefuly before continuing!

We will implement sum-product and max-sum and apply it to a poly-tree structured medical diagnosis example.

For this assignment you should use numpy ndarrays (constructed with np.array, np.zeros, np.ones, etc.). We need n-dimensional arrays in order to store conditional distributions with more than one conditioning variable. If you want to perform matrix multiplication on arrays, use the np.dot function; all infix operators including *, +, -, work element-wise on arrays.

## Part 0: Doing the math (5 points)
We start with a set of three Bernoulli distributed variables X, Y, Z. Calculate the marginals $p(Y=1)$ and $p(Y=0)$ using only the sum and product rule where,

$$
p(X=1) = 0.05 \\\\
p(Z=1) = 0.2 \\\\
$$
$$
p(Y = 1 | X = 1, Z = 1) = 0.99 \\\\
p(Y = 1 | X = 1, Z = 0) = 0.9 \\\\
p(Y = 1 | X = 0, Z = 1) = 0.7 \\\\
p(Y = 1 | X = 0, Z = 0) = 0.0001 \\\\
$$

While implementing the message passing algorithms you should be able to use the results of this question as a guidance.

YOUR ANSWER HERE
$p(Y=1) \\
= p(Y=1|X=1,Z=1) \cdot p(X=1) \cdot p(Z=1) + p(Y=1|X=0,Z=1) \cdot p(X=0) \cdot p(Z=1) + p(Y=1|X=1,Z=0) \cdot p(X=1) \cdot p(Z=0) + p(Y=1|X=0,Z=0) \cdot p(X=0) \cdot p(Z=0)      \\
= 0.99 \cdot 0.05 \cdot 0.2 + 0.7 \cdot 0.95 \cdot 0.2 + 0.9 \cdot 0.05 \cdot 0.8 + 0.0001 \cdot 0.95 \cdot 0.8      \\
= 0.178976 \\
\\
p(Y=0) = 1 - p(Y=1) = 0.821024$


## Part 1: The sum-product algorithm

We will implement a data structure to store a factor graph and to facilitate computations on this graph. Recall that a factor graph consists of two types of nodes, factors and variables. Below you will find some classes for these node types to get you started. Carefully inspect this code and make sure you understand what it does. Step by step will update its functionality.

In [1]:
%pylab inline

class Node(object):
    """
    Base-class for Nodes in a factor graph. Only instantiate sub-classes of Node.
    """
    def __init__(self, name):
        # A name for this Node, for printing purposes
        self.name = name
        
        # Neighbours in the graph, identified with their index in this list.
        # i.e. self.neighbours contains neighbour 0 through len(self.neighbours) - 1.
        self.neighbours = []
        
        # Reset the node-state (not the graph topology)
        self.reset()
        
    def reset(self):
        # Incomming messages; a dictionary mapping neighbours to messages.
        # That is, it maps  Node -> np.ndarray.
        self.in_msgs = {}
        
        # A set of neighbours for which this node has pending messages.
        # We use a python set object so we don't have to worry about duplicates.
        self.pending = set([])

    def add_neighbour(self, nb):
        self.neighbours.append(nb)

    def send_sp_msg(self, other):
        # To be implemented in subclass.
        raise Exception('Method send_sp_msg not implemented in base-class Node')
   
    def send_ms_msg(self, other):
        # To be implemented in subclass.
        raise Exception('Method send_ms_msg not implemented in base-class Node')
    
    def receive_msg(self, other, msg):
        # Store the incomming message, replacing previous messages from the same node
        self.in_msgs[other] = msg

        # TODO: add pending messages
        # self.pending.update(...)
    
    def __str__(self):
        # This is printed when using 'print node_instance'
        return self.name


class Variable(Node):
    def __init__(self, name, num_states):
        """
        Variable node constructor.
        Args:
            name: a name string for this node. Used for printing. 
            num_states: the number of states this variable can take.
            Allowable states run from 0 through (num_states - 1).
            For example, for a binary variable num_states=2,
            and the allowable states are 0, 1.
        """
        self.num_states = num_states
        
        # Call the base-class constructor
        super(Variable, self).__init__(name)
    
    def set_observed(self, observed_state):
        """
        Set this variable to an observed state.
        Args:
            observed_state: an integer value in [0, self.num_states - 1].
        """
        # Observed state is represented as a 1-of-N variable
        # Set all-but-one states to a very low probability;
        # (a bit of a hack to avoid -inf values when taking logs)
        self.observed_state[:] = 1e-10
        self.observed_state[observed_state] = 1.0
        
    def set_latent(self):
        """
        Erase an observed state for this variable and consider it latent again.
        """
        # No state is preferred, so set all entries of observed_state to 1.0
        # Using this representation we need not differentiate observed an latent
        # variables when sending messages.
        self.observed_state[:] = 1.0
        
    def reset(self):
        super(Variable, self).reset()
        self.observed_state = np.ones(self.num_states)
        
    def marginal(self):
        """
        Compute the marginal distribution of this Variable.
        It is assumed that message passing has completed when this function is called.
        """
        # TODO: compute marginal
        return None
    
    def unnormalized_log_marginal(self):
        """
        Compute the unnormalized log marginal distribution of this variable.
        It is assumed that message passing has completed when this function is called.
        """
        # TODO: compute unnormalized log marginal
        return None
    
    def send_sp_msg(self, other):
        # TODO: implement Variable -> Factor message for sum-product
        pass
   
    def send_ms_msg(self, other):
        # TODO: implement Variable -> Factor message for max-sum
        pass

class Factor(Node):
    def __init__(self, name, f, neighbours):
        """
        Factor node constructor.
        Args:
            name: a name string for this node. Used for printing
            f: a numpy.ndarray with N axes, where N is the number of neighbours.
               That is, the axes of f correspond to variables, and the index along that axes corresponds to a value of that variable.
               Each axis of the array should have as many entries as the corresponding neighbour variable has states.
            neighbours: a list of neighbouring Variables. Bi-directional connections are created.
        """
        # Call the base-class constructor
        super(Factor, self).__init__(name)
        
        f = np.array(f)  # For convenience, accept lists (of lists (of numbers or potentially lists of ...))
        
        assert len(neighbours) == f.ndim, 'Factor function f should accept as many arguments as this Factor node has neighbours'
        
        for nb_ind in range(len(neighbours)):
            nb = neighbours[nb_ind]
            assert f.shape[nb_ind] == nb.num_states, 'The range of the factor function f is invalid for input %i %s' % (nb_ind, nb.name)
            self.add_neighbour(nb)
            nb.add_neighbour(self)

        self.f = f
        
    def send_sp_msg(self, other):
        # TODO: implement Factor -> Variable message for sum-product
        pass
   
    def send_ms_msg(self, other):
        # TODO: implement Factor -> Variable message for max-sum
        pass


Populating the interactive namespace from numpy and matplotlib


### 1.1. Initialize the graph (5 points)
The equations in Part 0 can be represented by a factor graph. Instantiate this graph by creating Variable and Factor instances and linking them according to the graph structure. 
To instantiate the factor graph, first create the Variable nodes and then create Factor nodes, passing a list of neighbour Variables to each Factor. To get you started, we initialize the variable node for $X$ and the factor node corresponding to the prior $p(X)$.

In [2]:
X = Variable(name='X', num_states=2)
X_prior = Factor(name='p(X)',
                 f=np.array([0.95, 0.05]),
                 neighbours=[X])

# Please stick to the naming convention used below, otherwise the test functionality throughout the lab won't work
                                 
# YOUR CODE HERE
############### ############### ###############

Z = Variable(name='Z', num_states=2)
Z_prior = Factor(name='p(Z)',
                 f=np.array([0.8, 0.2]),
                 neighbours=[Z])
                 
Y = Variable(name='Y', num_states=2)
Y_cond = Factor(name='p(Y|X,Z)',     
                f=np.array([[[0.9999, 0.3], [0.1, 0.01]],
                            [[0.0001, 0.7], [0.9, 0.99]]]),
                neighbours=[Y, X, Z])

############### ############### ###############                                 


In [3]:
### Test test test
assert Z_prior.f.shape == (2,)
assert Y_cond.f.shape == (2, 2, 2)

We will be doing a lot of marginalizations, i.e. obtain the distribution of a *single* variable from a distribution over multiple variables. Let's first write a function `marginalize` for that.

In [4]:
# Write a function marginalize that given
def marginalize(P, dim):
    
    # YOUR CODE HERE
    ############### ############### ###############
    
    return np.sum(P,dim)

    ############### ############### ###############
    
# Lets try it
test_P = np.random.rand(2, 3, 4)
test_P = test_P / test_P.sum()  # Normalize for proper distribution

# Do the marginal distributions look like you expect? - Yes.
print (marginalize(test_P, 0))
print (marginalize(test_P, 1))
print (marginalize(test_P, 2))


[[0.05686793 0.12413429 0.10955681 0.10591064]
 [0.08425015 0.12342844 0.12265491 0.02126651]
 [0.05450952 0.0484591  0.07710894 0.07185277]]
[[0.11239342 0.10201807 0.18184459 0.08609187]
 [0.08323417 0.19400376 0.12747607 0.11293805]]
[[0.17950755 0.1637918  0.1390486 ]
 [0.21696212 0.1878082  0.11288172]]


### 1.2 Factor to variable messages (20 points)
Write a method `send_sp_msg(self, other)` for the Factor class, that checks if all the information required to pass a message to `Variable` `other` is present, computes the message and sends it to `other`. "Sending" here simply means calling the `receive_msg` function of the receiving node (we will implement this later). The message itself should be represented as a numpy array (np.array) whose length is equal to the number of states of the variable.

In the very end of 1.2 below we overwrite `send_sp_msg(self, other)` for the Factor class. In general, this is considered bad practise but in this lab it saves us from scrolling up and down all the time.

You will implement a function `send_sp_msg` that sends a message from a factor to a variable for the max-sum algorith. This function implements Equation 8.66 from Bishop. The message should be a numpy array (np.array) whose length is equal to the number of states of the variable.

It is a good idea to write a number of helper functions to implement these equations.

Since it is always a good idea to include checks, you should first write a method `can_send_message` that checks whether a node `node` has all the information required to pass a message to `other`. This should work for both variable and factor nodes.

In [5]:
def can_send_message(sender, receiver):
  
    # YOUR CODE HERE
    ############### ############### ###############

    # check if receiver is actual neighbour of sender
    if receiver not in sender.neighbours:
      return False 
          
    # check if all incoming messages existent
    for nb in sender.neighbours:
      if nb == receiver:
        continue
      else:
        if nb not in sender.in_msgs.keys():
          return False
        
    return True
  
    ############### ############### ###############

# Do the results make sense?
print (can_send_message(X, X_prior))
print (can_send_message(X_prior, X))
print (can_send_message(X_prior, Y_cond))

False
True
False



In Eq. 8.66, Bishop writes $f(x, x_1, ..., x_M)$, where $x$ corresponds to the variable that will receive the message. For now, assume this variable is the `index`-th neighbour of the factor. In order to ease implementation, it may be a good idea to write a function that rearanges the dimensions of `f` to comply with this notation, i.e. moves the dimension `index` to the front. Make sure to return a copy and keep all other dimensions in order! Use `np.moveaxis`.

In [6]:
def move_dimension_first(f, index):
  
    # YOUR CODE HERE
    ############### ############### ###############
    
    return np.moveaxis(f, index, 0)

    ############### ############### ###############

You should calculate the product of the incoming messages of all neighbours of the sending node except the receiving node. Therefore it may be useful to write a function `get_neighbour_messages` that gathers these messages in a list. If you want to not make things complicated, make sure the order of the messages in the list corresponds to the order of the variables in `neighbours`.

In [7]:
def get_neighbour_messages(sender, receiver):
  
    # YOUR CODE HERE
    ############### ############### ###############
    
    other_neighbours = [nb for nb in sender.neighbours if not nb == receiver]

    return [sender.in_msgs[nb] for nb in other_neighbours]

    ############### ############### ###############

Before marginalizing, we need to calculate $\prod_{m\in\text{ne}(f_s)\setminus x} \mu_{x_m\rightarrow f_s}(x_m)$ (Eq. 8.66) for all possible combinations of $x_1, ..., x_M$ (values of the neighbour nodes except the receiving node). An elegant and efficient way to calculate these is using the n-way outer product of vectors. This product takes n vectors $\mathbf{x}^{(1)}, \ldots, \mathbf{x}^{(n)}$ and computes a $n$-dimensional tensor (ndarray) whose element $i_0,i_1,...,i_n$ is given by $\prod_j \mathbf{x}^{(j)}_{i_j}$. In python, this is realized as `np.multiply.reduce(np.ix_(*vectors))` for a python list `vectors` of 1D numpy arrays. Try to figure out how this statement works -- it contains some useful functional programming techniques. What should `vectors` be? Try to see the link between the result and Eq. 8.66.

In [8]:
def calc_other_neighbour_msg_prod(sender, receiver):
  
    # YOUR CODE HERE
    ############### ############### ###############
    
    return np.multiply.reduce(np.ix_(*get_neighbour_messages(sender, receiver)))
    
    ############### ############### ###############

Following Eq. 8.66, before marginalizing, you should calculate the product of $f(x, x_1, ..., x_M)$ with $\prod_{m\in\text{ne}(f_s)\setminus x} \mu_{x_m\rightarrow f_s}(x_m)$ for all configurations of $x, x_1, ..., x_M$. Since the second part does not depend on $x$, its tensor representations are of different dimensions. You can overcome this problem by using a loop, but preferably use numpy broadcasting by first aligning the dimensions of the tensors. You can use `np.expand_dims` or `X[None, ...]` to insert one dimension at the front. Write a function `calculate_factor` that, given `f` (which is reordered such that $x$ corresponds to the first dimension) and the (outer) product of the other neighbour messages, computes $f(x, x_1, ..., x_M) \prod_{m\in\text{ne}(f_s)\setminus x} \mu_{x_m\rightarrow f_s}(x_m)$ for all configurations of $x, x_1, ..., x_M$.

In [9]:
def calculate_factor(f_neighb_first, neighbour_msg_prod):
  
    # YOUR CODE HERE
    ############### ############### ###############
    
    f_neighb_first = np.expand_dims(f_neighb_first, axis=0)
    
    return np.multiply(f_neighb_first, neighbour_msg_prod)

    ############### ############### ###############
  

Put all the pieces together to define a function `calc_sum_product_factor_to_variable_msg` that calculates Eq. 8.66.

In [10]:
def calc_sum_product_factor_to_variable_msg(factor, variable):
    
    # YOUR CODE HERE
    ############### ############### ###############

    f_neighb_first = move_dimension_first(factor.f, factor.neighbours.index(variable))

    neighbour_msg_prod = calc_other_neighbour_msg_prod(factor, variable)  

    sum_product_factor = calculate_factor(f_neighb_first, neighbour_msg_prod)

    while size(shape(sum_product_factor)) > 2:
      sum_product_factor = marginalize(sum_product_factor, -1)

    sum_product_factor = marginalize(sum_product_factor, 0)

    return sum_product_factor

    ############### ############### ###############

In [11]:
# Finally, we will define the send message function for you
def factor_send_sp_msg(self, variable):
    
    # YOUR CODE HERE
    ############### ############### ###############
    
    assert isinstance(variable, Variable), "Factor can only send messages to variable!"
    assert can_send_message(self, variable), "Cannot send message!"
    
    out_msg = calc_sum_product_factor_to_variable_msg(self, variable)
    
    ############### ############### ###############
    
    # Send the message
    variable.receive_msg(self, out_msg)    
    
    # Remove the pending sign if present
    self.pending.discard(variable)
    
Factor.send_sp_msg = factor_send_sp_msg


In [12]:
### Test test test
# message from X_prior to X
X_prior.reset()
X.reset()

X_prior.send_sp_msg(X)
assert np.allclose(list(X.in_msgs.values()), [0.95, 0.05])

# message from Z_prior to Z
Z_prior.reset()
Z.reset()

Z_prior.send_sp_msg(Z)
assert np.allclose(list(Z.in_msgs.values()), [0.8, 0.2])

# message from Y_cond to Y
Y_cond.reset()
Y.reset()

Y_cond.receive_msg(X, X_prior.f) # simulating that Y_cond received all necessary messages from X
Y_cond.receive_msg(Z, Z_prior.f) # simulating that Y_cond received all necessary messages from Z
Y_cond.send_sp_msg(Y)
assert np.allclose(list(Y.in_msgs.values()), [0.821024, 0.178976])


### 1.3 Variable to factor messages (10 points)

Write a method `calc_sum_product_variable_to_factor_msg(variable, factor)` that computes the message to be sent to a neighbour variable by a factor.

In [13]:
def calc_sum_product_variable_to_factor_msg(variable, factor):
    
    # YOUR CODE HERE
    ############### ############### ###############

    #check, how many neighbors it has -> if leaf node, return 1: TODO
    if len(variable.neighbours) == 1:
        neighbour_msg_prod = np.ones(variable.num_states)
    else:
 
        messages = get_neighbour_messages(variable, factor)
        neighbour_msg_prod = np.prod(messages, axis=0)

    return neighbour_msg_prod    

    ############### ############### ###############

In [14]:
# Finally, we will define the send message function for you
def variable_send_sp_msg(self, factor):
    
    # YOUR CODE HERE
    ############### ############### ###############
    
    assert isinstance(factor, Factor), "Variable can only send messages to factor!"
    assert can_send_message(self, factor), "Cannot send message!"
    
    out_msg = calc_sum_product_variable_to_factor_msg(self, factor)
    
    ############### ############### ###############
    
    # Send the message
    factor.receive_msg(self, out_msg)
    
    # Remove the pending sign if present
    self.pending.discard(factor)
    
Variable.send_sp_msg = variable_send_sp_msg


In [15]:
### Test test test
Y_cond.reset()
Y.reset()

# First message from X to Y_cond
X_prior.reset()
X.reset()
X_prior.send_sp_msg(X) # simulating that X received all necessary messages
X.send_sp_msg(Y_cond) 
assert np.allclose(list(Y_cond.in_msgs.values()), [0.95, 0.05])

# Second message from Z to Y_cond
Z_prior.reset()
Z.reset()
Z_prior.send_sp_msg(Z) # simulating that Z received all necessary messages
Z.send_sp_msg(Y_cond)
assert np.allclose(list(Y_cond.in_msgs.values()), [[0.95, 0.05], [0.8, 0.2]])


### Testing a single forward pass and a single backward pass
Before we go on we will make sure that messages can be passed in both directions the graph.

In [16]:
X_prior.reset()
X.reset()
Z_prior.reset()
Z.reset()
Y_cond.reset()
Y.reset()

# Forward pass
X_prior.send_sp_msg(X)
Z_prior.send_sp_msg(Z)
X.send_sp_msg(Y_cond)
Z.send_sp_msg(Y_cond)
Y_cond.send_sp_msg(Y)
assert np.allclose(list(Y.in_msgs.values()), [0.821024, 0.178976])

# Backward pass
Y.send_sp_msg(Y_cond)
Y_cond.send_sp_msg(X)
Y_cond.send_sp_msg(Z)
X.send_sp_msg(X_prior)
Z.send_sp_msg(Z_prior)
assert np.allclose(list(X.in_msgs.values()), [[0.95, 0.05],[1., 1.]])
assert np.allclose(list(Z.in_msgs.values()), [[0.8, 0.2],[1., 1.]])


### 1.4 Compute marginal (10 points)
Later in this assignment, we will implement message passing schemes to do inference. Once the message passing has completed, we will want to compute local marginals for each variable.
Write the method `marginal` for the `Variable` class, that computes a marginal distribution over that node.

In [17]:
def marginal(self):
    
    # YOUR CODE HERE
    ############### ############### ###############
    
    msg_prod = 1
    for key, value in self.in_msgs.items():
        msg_prod = msg_prod * value

    return msg_prod / msg_prod.sum()

    ############### ############### ###############     
      
Variable.marginal = marginal


In [18]:
### Test test test
# Simulate a single forward pass
X_prior.reset()
X.reset()
Z_prior.reset()
Z.reset()
Y_cond.reset()
Y.reset()

X_prior.send_sp_msg(X)
Z_prior.send_sp_msg(Z)
X.send_sp_msg(Y_cond)
Z.send_sp_msg(Y_cond)
Y_cond.send_sp_msg(Y)

assert np.allclose(X.marginal(), [0.95, 0.05])
assert np.allclose(Z.marginal(), [0.8, 0.2])
assert np.allclose(Y.marginal(), [0.821024, 0.178976])


### 1.5 Receiving messages (10 points)
In order to implement a message passing algorithms, we need some way to determine which nodes are ready to send messages to which neighbours. We make use of the concept of "pending messages", which is explained in Bishop (8.4.7): 
"we will say that a (variable or factor)
node a has a message pending on its link to a node b if node a has received any
message on any of its other links since the last time it sent a message to b. Thus,
when a node receives a message on one of its links, this creates pending messages
on all of its other links."

Before we say node a has a pending message for node b, we **must check that node a has received all messages needed to compute the message that is to be sent to b**.

Modify the function `receive_msg`, so that it updates the self.pending variable as described above. The member self.pending is a set that is to be filled with Nodes to which self has pending messages.

In [19]:
# ANSWER 1.5
def receive_msg(self, other, msg):
    self.in_msgs[other] = msg
    
    # YOUR CODE HERE
    ############### ############### ###############
    
    for nb in self.neighbours:
        if not nb == other and can_send_message(self, nb):
            self.pending.add(nb)  
        
    ############### ############### ###############
    
Node.receive_msg = receive_msg


In [20]:
### Test test test
X_prior.reset()
X.reset()
assert X_prior.pending == set()

X_prior.pending.add(X)
assert str(list(X_prior.pending)[0]) == X.name

X_prior.send_sp_msg(X)
assert X_prior.pending == set()


### 1.6 Inference Engine (10 points)
Write a function `sum_product(node_list)` that runs the sum-product message passing algorithm on a tree-structured factor graph with given nodes. The input parameter `node_list` is a list of all Node instances in the graph, which is assumed to be ordered correctly. That is, the list starts with a leaf node, which can always send a message. Subsequent nodes in `node_list` should be capable of sending a message when the pending messages of preceding nodes in the list have been sent. The sum-product algorithm then proceeds by passing over the list from beginning to end, sending all pending messages at the nodes it encounters. Then, in reverse order, the algorithm traverses the list again and again sends all pending messages at each node as it is encountered. For this to work, we initialized pending messages for all the leaf nodes, e.g. `X_prior.pending.add(X)`, where `X_prior` is a Factor node corresponding the the prior, `X` is a `Variable` node and the only connection of `X_prior` goes to `X`.

In [21]:
def sum_product(node_list):
    
    # YOUR CODE HERE
    ############### ############### ###############
    
    for idx, elem in enumerate(node_list):
        while elem.pending:        
            receiver = elem.pending.pop()
            elem.send_sp_msg(receiver)

    for idx, elem in enumerate(list(reversed(node_list))):
        while elem.pending:        
            receiver = elem.pending.pop()
            elem.send_sp_msg(receiver)
        
    ############### ############### ###############


In [22]:
### Test test test
nodes = [X_prior, X, Z_prior, Z, Y, Y_cond]
for n in nodes:
    n.reset()
    
X_prior.pending.add(X)
Z_prior.pending.add(Z)
Y.pending.add(Y_cond)

sum_product(nodes)
assert np.allclose(Y.marginal(), [0.821024, 0.178976])


### 1.7 Observed variables and probabilistic queries (15 points)
We will now use the inference engine to answer probabilistic queries. That is, we will set certain variables to observed values, and obtain the marginals over latent variables. We have already provided functions `set_observed` and `set_latent` that manage a member of Variable called `observed_state`. Modify the `calc_sum_product_variable_to_factor_msg` and `Variable.marginal` routines that you wrote before, to use `observed_state` so as to get the required marginals when some nodes are observed.

In [23]:
def calc_sum_product_variable_to_factor_msg(variable, factor):
    
    # YOUR CODE HERE
    ############### ############### ###############
    
    if len(variable.neighbours) == 1:
        neighbour_msg_prod = np.ones(variable.num_states)
    else:
        messages = get_neighbour_messages(variable, factor)
        neighbour_msg_prod = np.prod(messages, axis=0)
    
    return neighbour_msg_prod * variable.observed_state

    ############### ############### ###############

In [24]:
### Test, test, test
X_prior.reset()
X.reset()
Y_cond.reset()

X_prior.send_sp_msg(X)
X.set_observed(0)
X.send_sp_msg(Y_cond)

assert np.allclose(list(Y_cond.in_msgs.values()), [9.5e-01, 5.0e-12])


In [25]:
def marginal(self):
    
    # YOUR CODE HERE 
    ############### ############### ###############
    
    msg_prod = 1
    for key, value in self.in_msgs.items():
      msg_prod = msg_prod * value      
      
    msg_prod = msg_prod * self.observed_state
    return msg_prod / msg_prod.sum()

    ############### ############### ###############

Variable.marginal = marginal

In [26]:
### Test, test, test
# Simulate a single forward pass
X_prior.reset()
X.reset()
Z_prior.reset()
Z.reset()
Y_cond.reset()
Y.reset()

X.set_observed(0)
Z.set_observed(0)

X_prior.send_sp_msg(X)
Z_prior.send_sp_msg(Z)
X.send_sp_msg(Y_cond)
Z.send_sp_msg(Y_cond)
Y_cond.send_sp_msg(Y)

assert np.allclose(Y.marginal(), [9.99900000e-01, 1.00000022e-04])


## Part 2: The max-sum algorithm
Next, we implement the max-sum algorithm as described in section 8.4.5 of Bishop.

### 2.1 Factor to variable messages (10 points)
Implement the function `Factor.send_ms_msg` that sends Factor -> Variable messages for the max-sum algorithm. It is analogous to the `Factor.send_sp_msg` function you implemented before. Make sure it works for observed and unobserved nodes. Consider using a number of helper functions as seen in Part 1.

In [27]:
def factor_send_ms_msg(self, variable):
    
    ############### ############### ###############
    
    assert isinstance(variable, Variable), "Factor can only send messages to variable!"
    assert can_send_message(self, variable), "Cannot send message!"
    
    f_neighb_first = move_dimension_first(self.f, self.neighbours.index(variable))
    
    nb_message = get_neighbour_messages(self, variable)
    calc_other_neighbour_msg_sum = np.add.reduce(np.ix_(*nb_message)) + np.log(variable.observed_state)
    
    sum_msg = np.log(f_neighb_first) + calc_other_neighbour_msg_sum
    
    index = tuple(i for i in range(len(sum_msg.shape)) if i != 0)
    
    out_msg = np.max(sum_msg,index)
    
    ############### ############### ###############
    
    
    # Send the message
    variable.receive_msg(self, out_msg)
    
    # Remove the pending sign if present
    self.pending.discard(variable)
    
Factor.send_ms_msg = factor_send_ms_msg


In [28]:
### Test test test
# message from X_prior to X
X_prior.reset()
X.reset()

X_prior.send_ms_msg(X)
assert np.allclose(list(X.in_msgs.values()), [-0.05129329, -2.99573227])

# message from Z_prior to Z
Z_prior.reset()
Z.reset()

Z_prior.send_ms_msg(Z)
assert np.allclose(list(Z.in_msgs.values()), [-0.22314355, -1.60943791])

# message from Y_cond to Y
Y_cond.reset()
Y.reset()

Y_cond.receive_msg(X, X_prior.f) # simulating that Y_cond received all necessary messages from X
Y_cond.receive_msg(Z, Z_prior.f) # simulating that Y_cond received all necessary messages from Z
Y_cond.send_ms_msg(Y)
assert np.allclose(list(Y.in_msgs.values()), [1.74989999, 0.79332506])


### 2.2 Variable to factor messages (10 points)
Implement the `Variable.send_ms_msg` function that sends Variable -> Factor messages for the max-sum algorithm.

In [29]:
# ANSWER 2.2
def variable_send_ms_msg(self, factor):
    
    ############### ############### ###############
        
    if len(self.neighbours) == 1:
        # leaf variable-node
        out_msg = np.zeros(self.num_states)
    else:             
        nb_msg = get_neighbour_messages(self, factor) 
        out_msg = np.add.reduce(nb_msg)

    out_msg = out_msg + np.log(self.observed_state)
    
    ############### ############### ###############
    
    # Send the message
    factor.receive_msg(self, out_msg)
    
    # Remove the pending sign if present
    self.pending.discard(factor)

Variable.send_ms_msg = variable_send_ms_msg


In [30]:
### Test test test
Y_cond.reset()
Y.reset()

# First message from X to Y_cond
X_prior.reset()
X.reset()
X_prior.send_ms_msg(X) # simulating that X received all necessary messages
X.send_ms_msg(Y_cond)
assert np.allclose(list(Y_cond.in_msgs.values()), [-0.05129329, -2.99573227])

# Second message from Z to Y_cond
Z_prior.reset()
Z.reset()
Z_prior.send_ms_msg(Z) # simulating that Z received all necessary messages
Z.send_ms_msg(Y_cond)
assert np.allclose(list(Y_cond.in_msgs.values()), [[-0.05129329, -2.99573227], [-0.22314355, -1.60943791]])


### 2.3 Implement unnormalized log marginal (5 points)
Write the method `unnormalized_log_marginal` for the `Variable` class, that computes a unnormalized log marginal distribution over that node.

In [31]:
def unnormalized_log_marginal(self):
    
    # YOUR CODE HERE
    ############### ############### ###############

    msg_prod = 0
    for key, value in self.in_msgs.items():
        msg_prod += value
      
    msg_prod = msg_prod + np.log(self.observed_state) 
    
    ############### ############### ###############   
    
    return msg_prod

Variable.unnormalized_log_marginal = unnormalized_log_marginal


In [32]:
### Test test test
# Simulate a single forward pass
X_prior.reset()
X.reset()
Z_prior.reset()
Z.reset()
Y_cond.reset()
Y.reset()

X_prior.send_ms_msg(X)
Z_prior.send_ms_msg(Z)
X.send_ms_msg(Y_cond)
Z.send_ms_msg(Y_cond)
Y_cond.send_ms_msg(Y)

assert np.allclose(X.unnormalized_log_marginal(), [-0.05129329, -2.99573227])
assert np.allclose(Z.unnormalized_log_marginal(), [-0.22314355, -1.60943791])
assert np.allclose(Y.unnormalized_log_marginal(), [-0.27453685, -2.01740615])


### 2.4 Find a MAP state (10 points)

Using the same message passing schedule we used for sum-product, implement the max-sum algorithm. For simplicity, we will ignore issues relating to non-unique maxima. So there is no need to implement backtracking; the MAP state is obtained by a per-node maximization (eq. 8.98 in Bishop). Make sure your algorithm works with both latent and observed variables.

In [33]:
def max_sum(node_list):
    
    # YOUR CODE HERE
    ############### ############### ###############   
    
    for idx, elem in enumerate(node_list):
      while elem.pending:        
        receiver = elem.pending.pop()
        elem.send_ms_msg(receiver)
        
    for idx, elem in enumerate(list(reversed(node_list))):
      while elem.pending:        
        receiver = elem.pending.pop()
        elem.send_ms_msg(receiver)
    
    ############### ############### ###############   

In [34]:
### Test test test: unobserved
nodes = [X_prior, X, Z_prior, Z, Y_cond, Y]
for n in nodes:
    n.reset()
    
X_prior.pending.add(X)
Z_prior.pending.add(Z)
Y.pending.add(Y_cond)

max_sum(nodes)
assert np.allclose(Y.unnormalized_log_marginal(), [-0.27453685, -2.01740615] )


In [35]:
### Test test test: partiallY observed
nodes = [X_prior, X, Z_prior, Z, Y_cond, Y]
for n in nodes:
    n.reset()
    
X_prior.pending.add(X)
Z_prior.pending.add(Z)
Y.pending.add(Y_cond)

Z.set_observed(1)

max_sum(nodes)
assert np.allclose(Y.unnormalized_log_marginal(), [-2.86470401, -2.01740615])


Given the max-marginals what do you have to do in order to find the global optimum? Why can we neglect the normalization constant here?

YOUR ANSWER HERE

In order to find the global optimum, we need to keep track of the maximizing element when calculating the Factor-Variable messages (by using argmax() instead of max()). Then, once we know the most probable value of the final node $x_N$, we can follow this link (i.e. the maximizing element) to find the most probable state of node $x_{N-1}$ and so on back to the initial node $x_1$. This process is known as back-tracking and will results in a globally consistent maximizing configuration.

The normalization only gives us a continuous transformation and does not change the position of the maximum and can therefore be discarded.

## Part 3: Medical graph
Now that we implemented the sum-product and max-sum algorithm. We will apply them to a poly-tree structured medical diagnosis example.



### 3.1 Initialize the graph (5 points)

Convert the directed graphical model ("Bayesian Network") shown below to a factor graph. Instantiate this graph by creating Variable and Factor instances and linking them according to the graph structure. 
To instantiate the factor graph, first create the Variable nodes and then create Factor nodes, passing a list of neighbour Variables to each Factor.
Use the following prior and conditional probabilities.

$$
p(\verb+Influenza+) = 0.05 \\\\
p(\verb+Smokes+) = 0.2 \\\\
$$
$$
p(\verb+SoreThroat+ = 1 | \verb+Influenza+ = 1) = 0.3 \\\\
p(\verb+SoreThroat+ = 1 | \verb+Influenza+ = 0) = 0.001 \\\\
p(\verb+Fever+ = 1| \verb+Influenza+ = 1) = 0.9 \\\\
p(\verb+Fever+ = 1| \verb+Influenza+ = 0) 0.05 \\\\
p(\verb+Bronchitis+ = 1 | \verb+Influenza+ = 1, \verb+Smokes+ = 1) = 0.99 \\\\
p(\verb+Bronchitis+ = 1 | \verb+Influenza+ = 1, \verb+Smokes+ = 0) = 0.9 \\\\
p(\verb+Bronchitis+ = 1 | \verb+Influenza+ = 0, \verb+Smokes+ = 1) = 0.7 \\\\
p(\verb+Bronchitis+ = 1 | \verb+Influenza+ = 0, \verb+Smokes+ = 0) = 0.0001 \\\\
p(\verb+Coughing+ = 1| \verb+Bronchitis+ = 1) = 0.8 \\\\
p(\verb+Coughing+ = 1| \verb+Bronchitis+ = 0) = 0.07 \\\\
p(\verb+Wheezing+ = 1| \verb+Bronchitis+ = 1) = 0.6 \\\\
p(\verb+Wheezing+ = 1| \verb+Bronchitis+ = 0) = 0.001 \\\\
$$

In [36]:
# YOUR CODE HERE
############### ############### ###############

#create variables
Influenza = Variable(name='Influenza', num_states=2)
Smokes = Variable(name='Smokes', num_states=2)
SoreThroat = Variable(name='SoreThroat', num_states=2)
Fever = Variable(name='Fever', num_states=2)
Bronchitis = Variable(name='Bronchitis', num_states=2)
Coughing = Variable(name='Coughing', num_states=2)
Wheezing = Variable(name='Wheezing', num_states=2)


#create factors
Influenza_prior = Factor(name='p(Influenza)',
                        f=np.array([0.95,0.05]),
                        neighbours=[Influenza])

Smokes_prior = Factor(name='p(Smokes)',
                        f=np.array([0.8,0.2]),
                        neighbours=[Smokes])

SoreThroat_cond = Factor(name='p(SoreThroat|Influenza)',
                        f=np.array([[0.999,0.7],[0.001,0.3]]),
                        neighbours=[SoreThroat,Influenza])

Fever_cond = Factor(name='p(Fever|Influenza)',
                        f=np.array([[0.95,0.1],[0.05,0.9]]),
                        neighbours=[Fever,Influenza])

Bronchitis_cond = Factor(name='p(Bronchitis|Influenza,Smokes)',
                        f=np.array([[[0.9999,0.3],[0.1,0.01]],
                                    [[0.0001,0.7],[0.9,0.99]]]),
                        neighbours=[Bronchitis,Influenza,Smokes])

Coughing_cond = Factor(name='p(Coughing|Bronchitis)',
                        f=np.array([[0.93,0.2],[0.07,0.8]]),
                        neighbours=[Coughing,Bronchitis])

Wheezing_cond = Factor(name='p(Wheezing|Bronchitis)',
                        f=np.array([[0.999,0.4],[0.001,0.6]]),
                        neighbours=[Wheezing,Bronchitis])

############### ############### ###############


### 3.2. Sum-product algorithm (10 points)

3.2.1 Run the sum-product algorithm on an unobserved graph. Print the marginal for every variable node.

In [37]:
# YOUR CODE HERE
############### ############### ###############

nodes = [Influenza_prior, SoreThroat, SoreThroat_cond, Fever, Fever_cond, Influenza,
         Coughing, Coughing_cond, Wheezing, Wheezing_cond, Bronchitis, Smokes_prior,
         Smokes, Bronchitis_cond]

for n in nodes:
    n.reset()
    
Influenza_prior.pending.add(Influenza)
SoreThroat.pending.add(SoreThroat_cond)
Fever.pending.add(Fever_cond)
Coughing.pending.add(Coughing_cond)
Wheezing.pending.add(Wheezing_cond)
Smokes_prior.pending.add(Smokes)

sum_product(nodes)

for n in nodes:
  if type(n) == Variable:
    print(f'Marginal of {n} is {n.marginal()}')

############### ############### ###############

Marginal of SoreThroat is [0.98405 0.01595]
Marginal of Fever is [0.9075 0.0925]
Marginal of Influenza is [0.95 0.05]
Marginal of Coughing is [0.79934752 0.20065248]
Marginal of Wheezing is [0.89179338 0.10820662]
Marginal of Bronchitis is [0.821024 0.178976]
Marginal of Smokes is [0.8 0.2]


3.2.2 Rerun the sum-product algorithm on an partially observed graph, where the variable 'Influenza' is set to 0. Print the marginal for every variable node.

In [38]:
# YOUR CODE HERE
############### ############### ###############

for n in nodes:
    n.reset()
    
Influenza_prior.pending.add(Influenza)
SoreThroat.pending.add(SoreThroat_cond)
Fever.pending.add(Fever_cond)
Coughing.pending.add(Coughing_cond)
Wheezing.pending.add(Wheezing_cond)
Smokes_prior.pending.add(Smokes)

Influenza.set_observed(0)

sum_product(nodes)

for n in nodes:
  if type(n) == Variable:
    print(f'Marginal of {n} is {n.marginal()}')
    
############### ############### ###############

Marginal of SoreThroat is [0.999 0.001]
Marginal of Fever is [0.95 0.05]
Marginal of Influenza is [1.00000000e+00 5.26315789e-12]
Marginal of Coughing is [0.8277416 0.1722584]
Marginal of Wheezing is [0.91509208 0.08490792]
Marginal of Bronchitis is [0.85992 0.14008]
Marginal of Smokes is [0.8 0.2]


### 3.3. max-sum algorithm (10 points)

3.3.1 Run the max_sum algorithm on an unobserved graph. Print the marginal for every variable node.

In [39]:

# YOUR CODE HERE
############### ############### ###############
for n in nodes:
    n.reset()
    
Influenza_prior.pending.add(Influenza)
SoreThroat.pending.add(SoreThroat_cond)
Fever.pending.add(Fever_cond)
Coughing.pending.add(Coughing_cond)
Wheezing.pending.add(Wheezing_cond)
Smokes_prior.pending.add(Smokes)

max_sum(nodes)

for n in nodes:
  if type(n) == Variable:
    print(f'Unnormalized log-marginal of {n} is {n.unnormalized_log_marginal()}')

############### ############### ###############

Unnormalized log-marginal of SoreThroat is [-0.40040184 -5.36753884]
Unnormalized log-marginal of Fever is [-0.40040184 -3.34484082]
Unnormalized log-marginal of Influenza is [-0.40040184 -4.52024098]
Unnormalized log-marginal of Coughing is [-0.40040184 -2.80366912]
Unnormalized log-marginal of Wheezing is [-0.40040184 -2.80366912]
Unnormalized log-marginal of Bronchitis is [-0.40040184 -2.80366912]
Unnormalized log-marginal of Smokes is [-0.40040184 -2.80366912]


3.3.2 Rerun the max_sum algorithm on an partially observed graph, where the variable 'Influenza' is set to 0. Print the marginal for every variable node.

In [40]:

# YOUR CODE HERE
############### ############### ###############

for n in nodes:
    n.reset()
    
Influenza_prior.pending.add(Influenza)
SoreThroat.pending.add(SoreThroat_cond)
Fever.pending.add(Fever_cond)
Coughing.pending.add(Coughing_cond)
Wheezing.pending.add(Wheezing_cond)
Smokes_prior.pending.add(Smokes)

Influenza.set_observed(0)

max_sum(nodes)

for n in nodes:
  if type(n) == Variable:
    print(f'Unnormalized log-marginal of {n} is {n.unnormalized_log_marginal()}')

############### ############### ###############

Unnormalized log-marginal of SoreThroat is [-0.40040184 -7.30715662]
Unnormalized log-marginal of Fever is [-0.40040184 -3.34484082]
Unnormalized log-marginal of Influenza is [ -0.40040184 -52.76916741]
Unnormalized log-marginal of Coughing is [-0.40040184 -2.80366912]
Unnormalized log-marginal of Wheezing is [-0.40040184 -2.80366912]
Unnormalized log-marginal of Bronchitis is [-0.40040184 -2.80366912]
Unnormalized log-marginal of Smokes is [-0.40040184 -2.80366912]
